![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/nlu/blob/master/examples/colab/Training/binary_text_classification/NLU_training_sentiment_classifier_demo_twitter.ipynb)



# Training a Sentiment Analysis Classifier with NLU 
## 2 class twitter classifier training
With the [SentimentDL model](https://nlp.johnsnowlabs.com/docs/en/annotators#sentimentdl-multi-class-sentiment-analysis-annotator) from Spark NLP you can achieve State Of the Art results on any multi class text classification problem 

This notebook showcases the following features : 

- How to train the deep learning classifier
- How to store a pipeline to disk
- How to load the pipeline from disk (Enables NLU offline mode)



# 1. Install Java 8 and NLU

In [ ]:
!wget https://setup.johnsnowlabs.com/nlu/colab.sh -O - | bash
import nlu

--2021-05-05 11:33:05--  https://raw.githubusercontent.com/JohnSnowLabs/nlu/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1671 (1.6K) [text/plain]
Saving to: ‘STDOUT’

Installing  NLU 3.0.0 with  PySpark 3.0.2 and Spark NLP 3.0.1 for Google Colab ...
-                   100%[===================>]   1.63K  --.-KB/s    in 0s      

2021-05-05 11:33:06 (47.6 MB/s) - written to stdout [1671/1671]

     |████████████████████████████████| 204.8MB 67kB/s 
     |████████████████████████████████| 153kB 44.1MB/s 
     |████████████████████████████████| 204kB 22.3MB/s 
     |████████████████████████████████| 204kB 54.6MB/s 


# 2. Download twitter Sentiment dataset 
https://www.kaggle.com/cosmos98/twitter-and-reddit-sentimental-analysis-dataset
#Context

This is was a Dataset Created as a part of the university Project On Sentimental Analysis On Multi-Source Social Media Platforms using PySpark.

In [ ]:
! wget http://ckl-it.de/wp-content/uploads/2021/01/Twitter_Data.csv


--2021-05-05 11:35:08--  http://ckl-it.de/wp-content/uploads/2021/01/Twitter_Data.csv
Resolving ckl-it.de (ckl-it.de)... 217.160.0.108, 2001:8d8:100f:f000::209
Connecting to ckl-it.de (ckl-it.de)|217.160.0.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99657 (97K) [text/csv]
Saving to: ‘Twitter_Data.csv’

Twitter_Data.csv    100%[===================>]  97.32K   310KB/s    in 0.3s    

2021-05-05 11:35:08 (310 KB/s) - ‘Twitter_Data.csv’ saved [99657/99657]



In [ ]:
import pandas as pd
train_path = '/content/Twitter_Data.csv'

train_df = pd.read_csv(train_path)
# the text data to use for classification should be in a column named 'text'
# the label column must have name 'y' name be of type str
columns=['text','y']
train_df = train_df[columns]
train_df

,text,y
0,how narendra modi has almost killed the indian...,negative
1,you think was modi behind that accident,negative
2,kamal haasan takes chowkidar modi kamal haasan...,negative
3,connected name with surname not bcz religion c...,negative
4,anyone better than modi when nehruji expired s...,positive
...,...,...
595,perception makes fool some call âforeign inv...,negative
596,when will see your tweet for justice for you a...,negative
597,haha congress going gaga over this after looti...,positive
598,this movie shows the life histiry narendra mod...,negative


# 3. Train Deep Learning Classifier using nlu.load('train.sentiment')

You dataset label column should be named 'y' and the feature column with text data should be named 'text'

In [ ]:
from sklearn.metrics import classification_report
import nlu 
# load a trainable pipeline by specifying the train. prefix  and fit it on a datset with label and text columns
# by default the Universal Sentence Encoder (USE) Sentence embeddings are used for generation
trainable_pipe = nlu.load('train.sentiment')
fitted_pipe = trainable_pipe.fit(train_df.iloc[:50])

# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df.iloc[:50],output_level='document')
#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

preds

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
              precision    recall  f1-score   support

    negative       0.54      1.00      0.70        27
    positive       0.00      0.00      0.00        23

    accuracy                           0.54        50
   macro avg       0.27      0.50      0.35        50
weighted avg       0.29      0.54      0.38        50



,sentence_embedding_use,trained_sentiment,y,origin_index,text,trained_sentiment_confidence,document,sentence
0,"[0.060062434524297714, -0.05557167902588844, -...",negative,negative,0,how narendra modi has almost killed the indian...,0.781124,how narendra modi has almost killed the indian...,[how narendra modi has almost killed the india...
1,"[0.05362718552350998, -0.004547705873847008, -...",negative,negative,1,you think was modi behind that accident,0.772585,you think was modi behind that accident,[you think was modi behind that accident]
2,"[0.07274721562862396, -0.061593908816576004, -...",negative,negative,2,kamal haasan takes chowkidar modi kamal haasan...,0.778565,kamal haasan takes chowkidar modi kamal haasan...,[kamal haasan takes chowkidar modi kamal haasa...
3,"[0.06106054410338402, -0.060213156044483185, -...",negative,negative,3,connected name with surname not bcz religion c...,0.732274,connected name with surname not bcz religion c...,[connected name with surname not bcz religion ...
4,"[0.0737471729516983, 0.006071774289011955, -0....",negative,positive,4,anyone better than modi when nehruji expired s...,0.783452,anyone better than modi when nehruji expired s...,[anyone better than modi when nehruji expired ...
5,"[0.05888386443257332, -0.0646616667509079, -0....",negative,negative,5,\r\nmodiji wont tired crying foul\r\nmain chow...,0.788001,modiji wont tired crying foul main chowkidar h...,[modiji wont tired crying foul main chowkidar ...
6,"[0.058948416262865067, -0.029682165011763573, ...",negative,negative,6,poor chap modi hasnâ given him anything can ...,0.781363,poor chap modi hasnâ given him anything can ...,[poor chap modi hasnâ given him anything can...
7,"[0.051331546157598495, -0.06789953261613846, -...",negative,negative,7,green underwear missing ive been doubting isi ...,0.767519,green underwear missing ive been doubting isi ...,[green underwear missing ive been doubting isi...
8,"[0.044129759073257446, -0.06111813709139824, -...",negative,positive,8,congress years wasnt able complete one rafale ...,0.754464,congress years wasnt able complete one rafale ...,[congress years wasnt able complete one rafale...
9,"[0.03665374591946602, -0.03695330768823624, -0...",negative,negative,9,asked learn from how treat minority well does ...,0.693848,asked learn from how treat minority well does ...,[asked learn from how treat minority well does...


# Test the fitted pipe on new example

In [ ]:
fitted_pipe.predict('the president of india just died')

,sentence_embedding_use,trained_sentiment,origin_index,trained_sentiment_confidence,document,sentence
0,"[0.013345886953175068, -0.021778283640742302, ...",negative,0,0.722135,the president of india just died,[the president of india just died]


## Configure pipe training parameters

In [ ]:
trainable_pipe.print_info()

The following parameters are configurable for this NLU pipeline (You can copy paste the examples) :
>>> pipe['sentiment_dl'] has settable params:
pipe['sentiment_dl'].setMaxEpochs(1)                 | Info: Maximum number of epochs to train | Currently set to : 1
pipe['sentiment_dl'].setLr(0.005)                    | Info: Learning Rate | Currently set to : 0.005
pipe['sentiment_dl'].setBatchSize(64)                | Info: Batch size | Currently set to : 64
pipe['sentiment_dl'].setDropout(0.5)                 | Info: Dropout coefficient | Currently set to : 0.5
pipe['sentiment_dl'].setEnableOutputLogs(True)       | Info: Whether to use stdout in addition to Spark logs. | Currently set to : True
pipe['sentiment_dl'].setThreshold(0.6)               | Info: The minimum threshold for the final result otheriwse it will be neutral | Currently set to : 0.6
pipe['sentiment_dl'].setThresholdLabel('neutral')    | Info: In case the score is less than threshold, what should be the label. Default i

## Retrain with new parameters

In [ ]:
# Train longer!
trainable_pipe = nlu.load('train.sentiment')
trainable_pipe['trainable_sentiment_dl'].setMaxEpochs(5)  
fitted_pipe = trainable_pipe.fit(train_df.iloc[:50])
# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df.iloc[:50],output_level='document')

#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

preds

              precision    recall  f1-score   support

    negative       0.79      0.96      0.87        27
     neutral       0.00      0.00      0.00         0
    positive       1.00      0.09      0.16        23

    accuracy                           0.56        50
   macro avg       0.60      0.35      0.34        50
weighted avg       0.89      0.56      0.54        50



,sentence_embedding_use,trained_sentiment,y,origin_index,text,trained_sentiment_confidence,document,sentence
0,"[0.060062434524297714, -0.05557167902588844, -...",negative,negative,0,how narendra modi has almost killed the indian...,0.689142,how narendra modi has almost killed the indian...,[how narendra modi has almost killed the india...
1,"[0.05362718552350998, -0.004547705873847008, -...",negative,negative,1,you think was modi behind that accident,0.689483,you think was modi behind that accident,[you think was modi behind that accident]
2,"[0.07274721562862396, -0.061593908816576004, -...",negative,negative,2,kamal haasan takes chowkidar modi kamal haasan...,0.707988,kamal haasan takes chowkidar modi kamal haasan...,[kamal haasan takes chowkidar modi kamal haasa...
3,"[0.06106054410338402, -0.060213156044483185, -...",negative,negative,3,connected name with surname not bcz religion c...,0.675382,connected name with surname not bcz religion c...,[connected name with surname not bcz religion ...
4,"[0.0737471729516983, 0.006071774289011955, -0....",negative,positive,4,anyone better than modi when nehruji expired s...,0.638730,anyone better than modi when nehruji expired s...,[anyone better than modi when nehruji expired ...
5,"[0.05888386443257332, -0.0646616667509079, -0....",negative,negative,5,\r\nmodiji wont tired crying foul\r\nmain chow...,0.723110,modiji wont tired crying foul main chowkidar h...,[modiji wont tired crying foul main chowkidar ...
6,"[0.058948416262865067, -0.029682165011763573, ...",negative,negative,6,poor chap modi hasnâ given him anything can ...,0.690602,poor chap modi hasnâ given him anything can ...,[poor chap modi hasnâ given him anything can...
7,"[0.051331546157598495, -0.06789953261613846, -...",negative,negative,7,green underwear missing ive been doubting isi ...,0.705077,green underwear missing ive been doubting isi ...,[green underwear missing ive been doubting isi...
8,"[0.044129759073257446, -0.06111813709139824, -...",neutral,positive,8,congress years wasnt able complete one rafale ...,0.561979,congress years wasnt able complete one rafale ...,[congress years wasnt able complete one rafale...
9,"[0.03665374591946602, -0.03695330768823624, -0...",negative,negative,9,asked learn from how treat minority well does ...,0.746584,asked learn from how treat minority well does ...,[asked learn from how treat minority well does...


# Try training with different Embeddings

In [ ]:
# We can use nlu.print_components(action='embed_sentence') to see every possibler sentence embedding we could use. Lets use bert!
nlu.print_components(action='embed_sentence')

For language <en> NLU provides the following Models : 
nlu.load('en.embed_sentence') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.use') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.tfhub_use') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.use.lg') returns Spark NLP model tfhub_use_lg
nlu.load('en.embed_sentence.tfhub_use.lg') returns Spark NLP model tfhub_use_lg
nlu.load('en.embed_sentence.albert') returns Spark NLP model albert_base_uncased
nlu.load('en.embed_sentence.electra') returns Spark NLP model sent_electra_small_uncased
nlu.load('en.embed_sentence.electra_small_uncased') returns Spark NLP model sent_electra_small_uncased
nlu.load('en.embed_sentence.electra_base_uncased') returns Spark NLP model sent_electra_base_uncased
nlu.load('en.embed_sentence.electra_large_uncased') returns Spark NLP model sent_electra_large_uncased
nlu.load('en.embed_sentence.bert') returns Spark NLP model sent_bert_base_uncased
nlu.load('en.embed_sentenc

In [ ]:
trainable_pipe = nlu.load('en.embed_sentence.small_bert_L12_768 train.sentiment')
# We need to train longer and user smaller LR for NON-USE based sentence embeddings usually
# We could tune the hyperparameters further with hyperparameter tuning methods like gridsearch
# Also longer training gives more accuracy
trainable_pipe['trainable_sentiment_dl'].setMaxEpochs(100)  
trainable_pipe['trainable_sentiment_dl'].setLr(0.0005) 
fitted_pipe = trainable_pipe.fit(train_df)
# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df,output_level='document')

#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

#preds

sent_small_bert_L12_768 download started this may take some time.
Approximate size to download 392.9 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
              precision    recall  f1-score   support

    negative       0.78      0.67      0.72       300
     neutral       0.00      0.00      0.00         0
    positive       0.86      0.56      0.67       300

    accuracy                           0.61       600
   macro avg       0.54      0.41      0.46       600
weighted avg       0.82      0.61      0.70       600



# 5. Lets save the model

In [ ]:
stored_model_path = './models/classifier_dl_trained' 
fitted_pipe.save(stored_model_path)

Stored model in ./models/classifier_dl_trained


# 6. Lets load the model from HDD.
This makes Offlien NLU usage possible!   
You need to call nlu.load(path=path_to_the_pipe) to load a model/pipeline from disk.

In [ ]:
hdd_pipe = nlu.load(path=stored_model_path)

preds = hdd_pipe.predict('the president of india just died')
preds

,sentiment,sentence_embedding_from_disk,origin_index,text,sentiment_confidence,document,sentence
0,[positive],"[[0.009460609406232834, -0.07943306118249893, ...",8589934592,the president of india just died,[0.8165338],the president of india just died,[the president of india just died]


In [ ]:
hdd_pipe.print_info()

The following parameters are configurable for this NLU pipeline (You can copy paste the examples) :
>>> pipe['document_assembler'] has settable params:
pipe['document_assembler'].setCleanupMode('shrink')                                    | Info: possible values: disabled, inplace, inplace_full, shrink, shrink_full, each, each_full, delete_full | Currently set to : shrink
>>> pipe['sentence_detector@SentenceDetectorDLModel_c83c27f46b97'] has settable params:
pipe['sentence_detector@SentenceDetectorDLModel_c83c27f46b97'].setExplodeSentences(False)  | Info: whether to explode each sentence into a different row, for better parallelization. Defaults to false. | Currently set to : False
pipe['sentence_detector@SentenceDetectorDLModel_c83c27f46b97'].setStorageRef('SentenceDetectorDLModel_c83c27f46b97')  | Info: storage unique identifier | Currently set to : SentenceDetectorDLModel_c83c27f46b97
pipe['sentence_detector@SentenceDetectorDLModel_c83c27f46b97'].setEncoder(com.johnsnowlabs.nlp.anno